In [1]:
import pandas as pd

from twelvelabs import TwelveLabs
from twelvelabs.types import VideoSegment
from twelvelabs.embed import TasksStatusResponse

from twelvelabs.indexes import IndexesCreateRequestModelsItem
from twelvelabs.tasks import TasksRetrieveResponse

In [26]:
client = TwelveLabs(api_key=TL_API_KEY)

In [45]:
index = client.indexes.retrieve(index_id="68e1830166ecb2513d7eee5f")

In [46]:
# import requests

# url = "https://api.twelvelabs.io/v1.3/tasks"

# files = { "video_file": open('goal2-gk0xusut_YK88uF39.mp4', 'rb') }

In [47]:
payload = {
    "index_id": "68e1830166ecb2513d7eee5f",
}
headers = {"x-api-key": TL_API_KEY}

In [48]:
response = requests.post(url, data=payload, files=files, headers=headers)

print(response.json())

{'code': 'video_file_broken', 'message': 'Unable to process video file. Please check if the file is valid and try again.'}


In [127]:
result = client.gist(
    video_id=SPORTS_VIDEO_ID,
    types=["title", "topic", "hashtag"]
)
print("Result ID:", result.id)

if result.title is not None:
    print("Title:", result.title)

if result.topics is not None:
    print("Topics:")
    for topic in result.topics:
        print(f"  - {topic}")

if result.hashtags is not None:
    print("Hashtags:")
    for hashtag in result.hashtags:
        print(f"  - {hashtag}")

if result.usage is not None:
    print(f"Output tokens: {result.usage.output_tokens}")


Result ID: 5cfd4b94-815d-468f-9ed7-c5ea0b95ddca
Title: Number 7's Goal: A Celebratory Moment on the Soccer Field
Topics:
  - Soccer Game Highlights
Hashtags:
  - Soccer
  - Goal
  - Teamwork
  - Celebration
  - Uniforms
Output tokens: 30


In [130]:
from twelvelabs import TwelveLabs

response = client.analyze_stream(
    video_id=AD2_ID,
    prompt="Break down the video by main event and timestamp",
    temperature=0.2
)

for chunk in response:
    if hasattr(chunk, 'event_type'):
        if chunk.event_type == "stream_start":
            print("Stream started")
        elif chunk.event_type == "text_generation":
            print(chunk.text, end="")
        elif chunk.event_type == "stream_end":
            print("\nStream ended")
            if chunk.metadata:
                print(f"Metadata: {chunk.metadata}")


Stream started
The video can be broken down into the following main events with corresponding timestamps:

- **Application Begins (00:00 - 00:04):** The video starts with a close-up shot of a person's lips as a makeup brush with red lipstick begins to apply the product to the lower lip.
- **Continued Application (00:05 - 00:12):** The brush continues to apply the lipstick, moving from the center of the lower lip towards the outer edge.
- **Upper Lip Application (00:13 - 00:20):** The brush then moves to the upper lip, applying the lipstick from the center outwards.
- **Final Touches (00:21 - 00:24):** The brush applies the last touches of lipstick to the upper lip, completing the application process. The video ends with the lips fully coated in red lipstick.
Stream ended
Metadata: generation_id='e0e10b18-8036-4202-afaa-c03e9ed85970' usage=TokenUsage(output_tokens=193)


In [60]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.types import VideoSegment


In [80]:
def give_video_embedding(index_id,video_id):
    data = client.indexes.videos.retrieve(index_id, video_id, embedding_option=["visual-text", "audio"])

    return data.embedding.video_embedding.segments[0].float_

In [93]:
len(data.embedding.video_embedding.segments)

2

In [102]:

# 1. Retrieve the embeddings
video = client.indexes.videos.retrieve(index_id="<YOUR_INDEX_ID>", video_id="<YOUR_VIDEO_ID>", embedding_option=["visual-text"])


# 2. Process the results
def print_segments(segments: List[VideoSegment], max_elements: int = 5):
    for segment in segments:
        print(f"  embedding_scope={segment.embedding_scope} embedding_option={segment.embedding_option} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}")
        first_few = segment.float_[:max_elements]
        print(
            f"  embeddings: [{', '.join(str(x) for x in first_few)}...] (total: {len(segment.float_)} values)"
        )

print_segments(video.embedding.video_embedding.segments)


BadRequestError: headers: {'date': 'Sat, 04 Oct 2025 21:53:10 GMT', 'content-type': 'application/json; charset=UTF-8', 'content-length': '125', 'connection': 'keep-alive', 'tl-report': 'backend', 'vary': 'Accept-Encoding', 'x-trace-id': '4207796441159854403', 'strict-transport-security': 'max-age=31536000; includeSubDomains'}, status_code: 400, body: {'code': 'parameter_invalid', 'message': 'The index_id parameter is invalid. <YOUR_INDEX_ID> is not a valid id type'}

In [103]:
def get_video_embeddings(index_id,video_id):
    """
    Fetch segment embeddings for a video.
    Each segment embedding is a list of floats.
    """
    data = client.indexes.videos.retrieve(index_id, video_id, embedding_option=["visual-text"])

    #return data.embedding.video_embedding.segments[0].float_
#     url = f"{BASE_URL}/videos/{video_id}/embeddings"
#     resp = requests.get(url, headers=HEADERS)

#     if resp.status_code != 200:
#         raise Exception(f"Failed to fetch embeddings for {video_id}: {resp.text}")

    
    # Typically the response has a list of segments with embeddings and timestamps
    segments = []
    for seg in data.embedding.video_embedding.segments:
        embedding = np.array(seg.float_, dtype=np.float32)
        start_time = seg.start_offset_sec
        end_time = seg.end_offset_sec
        segments.append({"embedding": embedding, "start": start_time, "end": end_time})
    return segments

def get_video_level_embedding(index_id,video_id):
    """
    If ad video has no segments, average its embeddings.
    """
    segments = get_video_embeddings(index_id,video_id)
    if not segments:
        raise ValueError(f"No embeddings returned for video {video_id}")
    all_embs = np.stack([s["embedding"] for s in segments])
    return np.mean(all_embs, axis=0)

get_video_level_embedding("68e1830166ecb2513d7eee5f",SPORTS_VIDEO_ID)

array([ 0.03818677, -0.06685268,  0.03037306, ...,  0.02135889,
        0.03340793,  0.00982293], dtype=float32)

In [104]:
import os
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# Config
# ----------------------------
API_KEY = ""
BASE_URL = "https://api.twelvelabs.io/v1.3"  # Check docs for latest version

SPORTS_VIDEO_ID = "68e1843366ecb2513d7eee87"
AD1_ID = "68e1894217b39f617835d425"
AD2_ID = "68e188e43a1b0bed6c13561e"
INDEX_ID = "68e1830166ecb2513d7eee5f"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# ----------------------------
# Helper functions
# ----------------------------



def score_similarity(segment_emb, ad_emb):
    return float(cosine_similarity(segment_emb.reshape(1, -1), ad_emb.reshape(1, -1))[0][0])

# ----------------------------
# Main logic
# ----------------------------

sports_segments = get_video_embeddings(INDEX_ID,SPORTS_VIDEO_ID)
ad1_embedding = get_video_level_embedding(INDEX_ID,AD1_ID)
ad2_embedding = get_video_level_embedding(INDEX_ID,AD2_ID)


In [119]:
ads = {
        "Ad1": ad1_embedding,
        "Ad2": ad2_embedding,
    }


In [120]:
results = []
for seg in sports_segments:
    for ad_name, ad_emb in ads.items():
        score = score_similarity(seg["embedding"], ad_emb)
        results.append({
                "ad": ad_name,
                "start": seg["start"],
                "end": seg["end"],
                "engagement_score": round(score, 4)
            })


In [121]:
results = sorted(results, key=lambda x: x["engagement_score"], reverse=True)


In [122]:
print("\n🏆 Top Recommended Ad Placements:")
for r in results[:10]:
    print(f"🕐 {r['start']}s - {r['end']}s → {r['ad']} (Engagement: {r['engagement_score']})")



🏆 Top Recommended Ad Placements:
🕐 0.0s - 8.0s → Ad1 (Engagement: 0.3305)
🕐 0.0s - 8.0s → Ad2 (Engagement: 0.1572)


In [138]:
result = client.summarize(
    video_id=AD2_ID,
    type="summary",
    prompt="Summarise the video so that we put this AD in a video",
    temperature=0.2
)


In [139]:
result.summary

"The video showcases a meticulous and elegant application of red lipstick, capturing the essence of a beauty routine with a focus on the subtle artistry involved. The scene opens with a close-up of a person's lips, highlighting the precision of the application process. A hand, adorned with light pink nail polish, delicately holds a brush and applies the vibrant red lipstick to the lower lip. The camera's close focus allows viewers to appreciate the gradual transformation as the lipstick is evenly distributed, resulting in a striking and polished look. This video, with its attention to detail and the serene application process, serves as an ideal backdrop for an advertisement, emphasizing the allure and sophistication of the featured lipstick."

In [125]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")  # compact + fast

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
 player in a white uniform, wearing number 7, receives the ball from a teammate and scores a goal against a goalkeeper in green.After scoring, the player runs towards the corner flag, celebrating his goal by taking off his shirt and raising his arms in triumph. The camera captures his jubilant expression and movements as he acknowledges the crowd's cheers.

In [ ]:
The sneaker is shown from the side, focusing on the sole as it rotates to reveal the tread pattern.The camera angle shifts slightly to show the side profile of the shoe, highlighting the white color scheme and black laces.

In [ ]:
The video starts with a close-up shot of a person's lips as a makeup brush with red lipstick begins to apply the product to the lower lip.The brush applies the last touches of lipstick to the upper lip, completing the application process. The video ends with the lips fully coated in red lipstick